# The beginnings of Bernie the Bot

In [10]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_curve, auc
from prophet_rmse import stock_rmse
import seaborn as sns
from datetime import datetime, timezone, timedelta

In [11]:
class Bernie:
    def __init__(self):
        self.balance = 20000
        self.ticker_list = ['PNM','GALT','GAIA','GLYC']
        self.portfolio = {'date': [], 'ticker': [], 'b/s': [], 'price': [], 'quantity': []}
        self.date = '2015-01-01'

    def predict_tomorrow(self):
        self.tom_forecast = []

        for t in self.ticker_list:
            raw_data = yf.download(t, start="2010-01-01", end=self.date, interval='1d')

            df = pd.DataFrame({'ds': raw_data.index, 'y': raw_data['Adj Close']})
            df.reset_index(inplace=True, drop=True)

            m = Prophet(daily_seasonality=True)

            m.fit(df)
            future = m.make_future_dataframe(periods=3)
            forecast = m.predict(future)

            n_three = forecast[-4:].yhat
            n_three = n_three.tolist()

            self.tom_forecast.append((t, n_three))

    def check_buy(self):
        for i in range(4):
            t = self.tom_forecast[i][0]
            n_3 = self.tom_forecast[i][1]

            if (n_3[0] < n_3[1]) and (n_3[1] < n_3[2]) and (n_3[2] < n_3[3]):
                self.buy(t)

    def check_sell(self):
        for i in range(4):
            n_3 = self.tom_forecast[i][1]
            t = self.tom_forecast[i][0]

            print(n_3[0], n_3[1])

            if n_3[0] > n_3[1]:
                self.sell(t)

    def buy(self, ticker):
        quantity = 5

        self.portfolio['date'].append(self.date)
        self.portfolio['ticker'].append(ticker)
        self.portfolio['b/s'].append('b')
        self.portfolio['price'].append((self.tom_forecast[1][1][0]) * quantity)
        self.portfolio['quantity'].append(quantity)

        self.balance -= (self.tom_forecast[1][1][0]) * quantity

    def sell(self, ticker):
        quantity = 5

        self.portfolio['date'].append(self.date)
        self.portfolio['ticker'].append(ticker)
        self.portfolio['b/s'].append('s')
        self.portfolio['price'].append((self.tom_forecast[1][1][0]) * quantity)
        self.portfolio['quantity'].append(quantity)

        self.balance += (self.tom_forecast[1][1][0]) * quantity

    def increment_day(self):
        d = datetime.strptime(self.date, '%Y-%m-%d')
        d = d + timedelta(days=1)
        self.date = d.strftime('%Y-%m-%d')


In [12]:
bernie = Bernie()
while bernie.date != '2016-01-01':
    bernie.predict_tomorrow()
    bernie.check_buy()
    bernie.check_sell()
    bernie.increment_day()
    print(bernie.date, bernie.balance)

print(bernie.balance)

[*********************100%***********************]  1 of 1 completed


14:59:10 - cmdstanpy - INFO - Chain [1] start processing
14:59:12 - cmdstanpy - INFO - Chain [1] done processing


[*********************100%***********************]  1 of 1 completed


14:59:14 - cmdstanpy - INFO - Chain [1] start processing
14:59:17 - cmdstanpy - INFO - Chain [1] done processing


[*********************100%***********************]  1 of 1 completed


14:59:18 - cmdstanpy - INFO - Chain [1] start processing


KeyboardInterrupt: 

In [ ]:
bernie.tom_forecast[1][0]